<a href="https://colab.research.google.com/github/NavyaSrija/Brain-Tumor-Segmentation-and-Classification-Using-MRI-and-CT-Imaging/blob/main/MRI_CT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CT** **classification**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

DATA_ROOT = '/content/drive/MyDrive/Dataset of CTMRI'


In [ ]:
for root, dirs, files in os.walk(DATA_ROOT):
    print(f"In: {root}")
    print("  Subfolders:", dirs)
    print("  Files:", files[:5])  # Show up to 5 files
    print('-'*50)


In: /content/drive/MyDrive/Dataset of CTMRI
  Subfolders: ['Dataset']
  Files: []
--------------------------------------------------
In: /content/drive/MyDrive/Dataset of CTMRI/Dataset
  Subfolders: ['Brain Tumor MRI images', 'Brain Tumor CT scan Images']
  Files: []
--------------------------------------------------
In: /content/drive/MyDrive/Dataset of CTMRI/Dataset/Brain Tumor MRI images
  Subfolders: ['Healthy', 'Tumor', 'Tumor_masks']
  Files: ['MRI image source.txt']
--------------------------------------------------
In: /content/drive/MyDrive/Dataset of CTMRI/Dataset/Brain Tumor MRI images/Healthy
  Subfolders: []
  Files: ['mri_healthy (1618).jpg', 'mri_healthy (279).jpg', 'mri_healthy (1533).jpg', 'mri_healthy (1854).jpg', 'mri_healthy (1355).jpg']
--------------------------------------------------
In: /content/drive/MyDrive/Dataset of CTMRI/Dataset/Brain Tumor MRI images/Tumor
  Subfolders: []
  Files: ['pituitary (187).jpg', 'meningioma (823).jpg', 'pituitary (425).jpg', 'me

In [ ]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

ct_tumor_dir = '/content/drive/MyDrive/Dataset of CTMRI/Dataset/Brain Tumor CT scan Images/Tumor'
ct_healthy_dir = '/content/drive/MyDrive/Dataset of CTMRI/Dataset/Brain Tumor CT scan Images/Healthy'

ct_data = []
for fname in os.listdir(ct_tumor_dir):
    if fname.endswith(('.jpg', '.png')):
        ct_data.append((os.path.join(ct_tumor_dir, fname), 1))  # 1 = tumor
for fname in os.listdir(ct_healthy_dir):
    if fname.endswith(('.jpg', '.png')):
        ct_data.append((os.path.join(ct_healthy_dir, fname), 0))  # 0 = healthy

# (You can repeat the same for MRI if you want to compare across modalities!)

train_list, val_list = train_test_split(ct_data, test_size=0.2, random_state=42)

class BrainTumorClassificationDataset(Dataset):
    def __init__(self, img_label_list, resize=(224,224)):
        self.img_label_list = img_label_list
        self.resize = resize
    def __len__(self):
        return len(self.img_label_list)
    def __getitem__(self, idx):
        img_path, label = self.img_label_list[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, self.resize)
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)
        img = np.stack([img]*3, axis=0)
        img = torch.tensor(img, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.long)
        return img, label

train_ds = BrainTumorClassificationDataset(train_list)
val_ds = BrainTumorClassificationDataset(val_list)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)


In [ ]:
import timm
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# EfficientNet
model_eff = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
model_eff = model_eff.to(device)

# ConvNeXt Tiny (latest conv-based model)
model_cnv = timm.create_model('convnext_tiny', pretrained=True, num_classes=2)
model_cnv = model_cnv.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

In [ ]:
import timm
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# EfficientNet
model_eff = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
model_eff = model_eff.to(device)

# ConvNeXt Tiny (latest conv-based model)
model_cnv = timm.create_model('convnext_tiny', pretrained=True, num_classes=2)
model_cnv = model_cnv.to(device)


In [ ]:
def train_model(model, train_loader, val_loader, epochs=2):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}: {epoch_loss / len(train_loader):.4f}")
    return model


In [ ]:
print("\nTraining EfficientNetB0...")
model_eff = train_model(model_eff, train_loader, val_loader)

print("\nTraining ConvNeXt Tiny...")
model_cnv = train_model(model_cnv, train_loader, val_loader)



Training EfficientNetB0...
Epoch 1: 0.3875
Epoch 2: 0.0399

Training ConvNeXt Tiny...
Epoch 1: 0.2295
Epoch 2: 0.1366


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def eval_model(model, val_loader, name="Model"):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    acc = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=['Healthy','Tumor'], digits=4)
    print(f"\n{name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(report)
    return acc, report

eval_model(model_eff, val_loader, "EfficientNetB0")
eval_model(model_cnv, val_loader, "ConvNeXt Tiny")



EfficientNetB0 Results:
Accuracy: 0.9805
              precision    recall  f1-score   support

     Healthy     0.9774    0.9819    0.9796       441
       Tumor     0.9834    0.9793    0.9813       483

    accuracy                         0.9805       924
   macro avg     0.9804    0.9806    0.9805       924
weighted avg     0.9805    0.9805    0.9805       924


ConvNeXt Tiny Results:
Accuracy: 0.9232
              precision    recall  f1-score   support

     Healthy     0.9602    0.8753    0.9158       441
       Tumor     0.8946    0.9669    0.9294       483

    accuracy                         0.9232       924
   macro avg     0.9274    0.9211    0.9226       924
weighted avg     0.9259    0.9232    0.9229       924



(0.9231601731601732,
 '              precision    recall  f1-score   support\n\n     Healthy     0.9602    0.8753    0.9158       441\n       Tumor     0.8946    0.9669    0.9294       483\n\n    accuracy                         0.9232       924\n   macro avg     0.9274    0.9211    0.9226       924\nweighted avg     0.9259    0.9232    0.9229       924\n')

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def eval_model(model, val_loader, name="Model"):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    acc = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=['Healthy','Tumor'], digits=4)
    print(f"\n{name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(report)
    return acc, report

eval_model(model_eff, val_loader, "EfficientNetB0")
eval_model(model_cnv, val_loader, "ConvNeXt Tiny")



EfficientNetB0 Results:
Accuracy: 0.9805
              precision    recall  f1-score   support

     Healthy     0.9774    0.9819    0.9796       441
       Tumor     0.9834    0.9793    0.9813       483

    accuracy                         0.9805       924
   macro avg     0.9804    0.9806    0.9805       924
weighted avg     0.9805    0.9805    0.9805       924


ConvNeXt Tiny Results:
Accuracy: 0.9232
              precision    recall  f1-score   support

     Healthy     0.9602    0.8753    0.9158       441
       Tumor     0.8946    0.9669    0.9294       483

    accuracy                         0.9232       924
   macro avg     0.9274    0.9211    0.9226       924
weighted avg     0.9259    0.9232    0.9229       924



(0.9231601731601732,
 '              precision    recall  f1-score   support\n\n     Healthy     0.9602    0.8753    0.9158       441\n       Tumor     0.8946    0.9669    0.9294       483\n\n    accuracy                         0.9232       924\n   macro avg     0.9274    0.9211    0.9226       924\nweighted avg     0.9259    0.9232    0.9229       924\n')

**MRI classification**

In [ ]:
import os

# MRI paths (update if different)
mri_tumor_dir = '/content/drive/MyDrive/Dataset of CTMRI/Dataset/Brain Tumor MRI images/Tumor'
mri_healthy_dir = '/content/drive/MyDrive/Dataset of CTMRI/Dataset/Brain Tumor MRI images/Healthy'

mri_data = []
for fname in os.listdir(mri_tumor_dir):
    if fname.endswith('.jpg'):
        mri_data.append((os.path.join(mri_tumor_dir, fname), 1))  # 1 = tumor
for fname in os.listdir(mri_healthy_dir):
    if fname.endswith('.jpg'):
        mri_data.append((os.path.join(mri_healthy_dir, fname), 0))  # 0 = healthy

from sklearn.model_selection import train_test_split
mri_train, mri_val = train_test_split(mri_data, test_size=0.2, random_state=42)


In [ ]:
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class BrainTumorMRIDataset(Dataset):
    def __init__(self, img_label_list, resize=(224,224)):
        self.img_label_list = img_label_list
        self.resize = resize
    def __len__(self):
        return len(self.img_label_list)
    def __getitem__(self, idx):
        img_path, label = self.img_label_list[idx]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, self.resize)
        img = (img - np.min(img)) / (np.max(img) - np.min(img) + 1e-8)
        img = np.stack([img]*3, axis=0)
        img = torch.tensor(img, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.long)
        return img, label

mri_train_ds = BrainTumorMRIDataset(mri_train)
mri_val_ds = BrainTumorMRIDataset(mri_val)
mri_train_loader = DataLoader(mri_train_ds, batch_size=32, shuffle=True)
mri_val_loader = DataLoader(mri_val_ds, batch_size=32, shuffle=False)


In [ ]:
import timm
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# EfficientNetB0
mri_model_eff = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
mri_model_eff = mri_model_eff.to(device)

# ConvNeXt Tiny
mri_model_cnv = timm.create_model('convnext_tiny', pretrained=True, num_classes=2)
mri_model_cnv = mri_model_cnv.to(device)


In [ ]:
def train_model(model, train_loader, val_loader, epochs=2):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}: {epoch_loss / len(train_loader):.4f}")
    return model


In [ ]:
print("\nTraining EfficientNetB0 on MRI...")
mri_model_eff = train_model(mri_model_eff, mri_train_loader, mri_val_loader)

print("\nTraining ConvNeXt Tiny on MRI...")
mri_model_cnv = train_model(mri_model_cnv, mri_train_loader, mri_val_loader)



Training EfficientNetB0 on MRI...
Epoch 1: 0.3928
Epoch 2: 0.0222

Training ConvNeXt Tiny on MRI...
Epoch 1: 0.2349
Epoch 2: 0.0341


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

def eval_model(model, val_loader, name="Model"):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    acc = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=['Healthy','Tumor'], digits=4)
    print(f"\n{name} Results:")
    print(f"Accuracy: {acc:.4f}")
    print(report)
    return acc, report

eval_model(mri_model_eff, mri_val_loader, "EfficientNetB0 (MRI)")
eval_model(mri_model_cnv, mri_val_loader, "ConvNeXt Tiny (MRI)")



EfficientNetB0 (MRI) Results:
Accuracy: 0.9890
              precision    recall  f1-score   support

     Healthy     0.9900    0.9826    0.9863       403
       Tumor     0.9883    0.9933    0.9908       594

    accuracy                         0.9890       997
   macro avg     0.9891    0.9879    0.9885       997
weighted avg     0.9890    0.9890    0.9890       997


ConvNeXt Tiny (MRI) Results:
Accuracy: 0.9689
              precision    recall  f1-score   support

     Healthy     0.9745    0.9479    0.9610       403
       Tumor     0.9653    0.9832    0.9741       594

    accuracy                         0.9689       997
   macro avg     0.9699    0.9655    0.9676       997
weighted avg     0.9690    0.9689    0.9688       997



(0.9689067201604814,
 '              precision    recall  f1-score   support\n\n     Healthy     0.9745    0.9479    0.9610       403\n       Tumor     0.9653    0.9832    0.9741       594\n\n    accuracy                         0.9689       997\n   macro avg     0.9699    0.9655    0.9676       997\nweighted avg     0.9690    0.9689    0.9688       997\n')